In [2]:
import os
import torch

In [3]:
ckpt = torch.load(os.path.join('/f_data/G', "Emu/Emu/Emu-instruct.pt"), map_location="cpu")

In [88]:
new_state_dicts = OrderedDict()
    
# directly load, visual, ln_visual, cformer
for k, v in ckpt.items():
    if 'decoder.lm.base_model.model.model' in k: # embed_tokens, layers
        new_state_dicts[k.replace('decoder.lm.base_model.model.model', 'base_model.model.model')] = v
    elif 'decoder.lm.base_model.model.lm_head' in k: # lm_head
        new_state_dicts[k.replace('decoder.lm.base_model.model.lm_head', 'base_model.model.lm_head')] = v
    elif 'decoder.lm.base_model.model.stu_regress_head.weight' in k: # stu_regress_head
        new_state_dicts['base_model.model.stu_regress_head.weight'] = v
    else:
        new_state_dicts['base_model.model.'+k] = v

In [95]:
import sys
sys.path.append('/workspace/Llama-X')
from src.train_emu_t2i import *
from transformers.models.llama.configuration_llama import LlamaConfig

In [96]:
args = llamaconfig()

config = LlamaConfig.from_pretrained('/f_data/G/llama/llama-13b-hf/')
emu_config = file2data('/workspace/Llama-X/emu/Emu-14B.json')

Loaded data from /workspace/Llama-X/emu/Emu-14B.json


In [97]:
modelArguments = ModelArguments(model_name_or_path='/f_data/G/llama/llama-13b-hf/')
dataArguments = DataArguments()
trainingArguments = TrainingArguments(output_dir=None)

In [98]:
model = LlamaNUWA(config=config, **emu_config, cast_dtype=torch.float16)

Init Image Encoder


In [99]:
IGNORE_INDEX = -100
DEFAULT_BOS_TOKEN = '<s>'
DEFAULT_EOS_TOKEN = '</s>'
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_UNK_TOKEN = "<unk>"

DEFAULT_IMG_START_TOKEN = "[IMG]"
DEFAULT_IMG_END_TOKEN = "[/IMG]"
DEFAULT_IMG_TOKEN = "<image>"
USER_TOKEN = '[USER]'
ASSISTANT_TOKEN = '[ASSISTANT]'
tokenizer = transformers.LlamaTokenizer.from_pretrained(
        '/f_data/G/llama/llama-13b-hf/',
        model_max_length=512,
        padding_side="right",
        use_fast=True,
    )
    
special_token_list = [DEFAULT_IMG_START_TOKEN, DEFAULT_IMG_END_TOKEN, DEFAULT_IMG_TOKEN, USER_TOKEN,
                              ASSISTANT_TOKEN]
special_tokens_dict = dict(
        pad_token=DEFAULT_PAD_TOKEN,
        bos_token=DEFAULT_BOS_TOKEN,
        eos_token=DEFAULT_EOS_TOKEN,
        unk_token=DEFAULT_UNK_TOKEN,
        additional_special_tokens=special_token_list
    )
if tokenizer.pad_token is None:
    smart_tokenizer_and_embedding_resize(
        special_tokens_dict=special_tokens_dict,
        tokenizer=tokenizer,
        model=model,
    )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Using pad_token, but it is not set yet.


In [100]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

In [101]:
adaptively_load_state_dict(model, new_state_dicts)

Some weights of state_dict were not used in target: ['base_model.model.stu_regress_head.weight']


In [104]:
model.state_dict().keys()

odict_keys(['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.rotary_emb.inv_freq', 'base_model.model.model.layers.0.mlp.gate_proj.we

In [109]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaNUWA(
      (model): LlamaModel(
        (embed_tokens): Embedding(32006, 5120)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): Modu

In [112]:
model.model.model.embed_tokens(torch.LongTensor([0]))

tensor([[ 0.0002, -0.0001,  0.0008,  ...,  0.0008, -0.0005,  0.0006]])

In [114]:
model.base_model.model.model.embed_tokens(torch.LongTensor([0]))

tensor([[ 0.0002, -0.0001,  0.0008,  ...,  0.0008, -0.0005,  0.0006]])

In [107]:
model.state_dict()['model.embed_tokens.weight'].shape

KeyError: 'model.embed_tokens.weight'

In [83]:
new_state_dicts['base_model.model.model.embed_tokens.weight'].shape

torch.Size([32006, 5120])

In [ ]:
# Shape not Align